In [1]:
!TMPDIR=/ pip install --upgrade "pymilvus[model]" openai requests tqdm sentence-transformers transformers text2vec dashscope

Looking in indexes: http://mirrors.cloud.aliyuncs.com/pypi/simple/


In [2]:
# 在最开头执行
import os

# 设置HuggingFace镜像站
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"   
os.environ["HUGGINGFACE_CO_URL_HOME"] = "https://hf-mirror.com"

In [3]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY_K1")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY_K1 环境变量")

# 初始化客户端
client = OpenAI(
    api_key="anything",
    base_url="http://117.50.173.79:11434/v1",
)

## 2. 需求拆解与Agent任务规划

#### 用户痛点与核心需求：
*   **效率低下：** 人工创作周期长，难以满足高频发布需求。
*   **创意瓶颈：** 难以持续产出新颖、吸引人的爆款创意。
*   **趋势捕捉难：** 实时流行元素难以快速融入文案。
*   **平台特性把握：** 小红书特有风格（标题、正文、标签、表情）难以精准复制。

#### “爆款”文案的特征：
*   **强吸引力标题：** 制造好奇、痛点共鸣、利益点清晰。
*   **沉浸式正文：** 真实体验分享、细节描述、情感共鸣。
*   **精准且多样标签：** 热门话题、品牌词、产品词、垂直领域词。
*   **生动表情符号：** 增强表达力，提升活泼感。
*   **清晰的行动召唤 (CTA)。**

#### Agent 任务规划：核心工作流
1.  **用户指令接收：** 接收产品信息、主题、风格等。
2.  **信息收集 (Web Search/DB Query)：** 实时搜索行业趋势、热门话题、竞品分析、产品卖点。
3.  **内容构思与初稿生成 (LLM)：** 结合所有信息，撰写标题、正文、标签、表情符号。
4.  **风格与格式优化 (LLM)：** 根据小红书平台特点和指定风格，对文案进行润色和结构调整。
5.  **最终输出：** 呈现完整文案。

## 3. 爆款文案生成逻辑与 Prompt 设计

### 3.1 System Prompt (系统提示词)

In [4]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
"""

In [5]:
# 初始化产品数据至向量数据库

# 定义向量化模型
# from pymilvus import model as milvus_model
# embedding_model = milvus_model.DefaultEmbeddingFunction()
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 创建Collection
from pymilvus import MilvusClient
milvus_client = MilvusClient(uri="./milvus_products.db")
collection_name = "products"

# 为支持重复执行代码，此处每次执行时都先删除Collection
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

# 使用测试文本进行向量化后获取向量维度，用于创建Collection时设置维度参数
test_embedding = embedding_model.encode("This is a test")
embedding_dim = len(test_embedding)

# 创建Collection
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong"
)

# 定义商品数据集
products = [
    "黑巧克力（85%可可）：深褐色光滑块状，微苦带甜，富含抗氧化剂，适合需要提神和控糖的人群。",
    "蓝莓：小巧深蓝色浆果，酸甜多汁，含丰富花青素，适合护眼和抗衰老需求者。",
    "牛油果：梨形墨绿色果实，奶油般绵密口感，高含健康脂肪酸，适合健身增肌人群。",
    "三文鱼刺身：橘红色鲜嫩鱼片，冰凉滑嫩，富含Omega-3，适合心脑血管保养者。",
    "希腊酸奶：浓稠乳白色凝乳，微酸醇厚，高蛋白低糖，适合减脂塑形人群。",
    "奇亚籽：芝麻状黑色颗粒，遇水膨胀无味，高膳食纤维，适合便秘调理者。",
    "巴西坚果：月牙形乳白果仁，油脂香浓，含超高硒元素，适合免疫力低下者。",
    "姜黄粉：亮黄色细粉末，温热微苦，强抗炎特性，适合关节养护人群。",
    "羽衣甘蓝脆片：深绿卷曲叶片，咸香酥脆，维生素K含量高，适合缺钙人群。",
    "椰子水：透明清亮液体，清甜爽口，电解质丰富，适合运动后补水。",
    "藜麦：浅黄色小圆粒，坚果风味，完全蛋白质来源，适合素食者。",
    "蔓越莓干：暗红色皱缩果干，酸甜带涩，预防尿路感染，适合女性保健。",
    "沙棘原浆：橙红色浓稠液体，酸涩回甘，VC含量超高，适合美白需求者。",
    "鹰嘴豆泥：浅褐绵密膏状，豆香浓郁，低GI高蛋白，适合糖尿病人群。",
    "黑蒜：深褐软糯蒜瓣，甜酸无刺激，含硫化合物，适合三高调理者。",
    "抹茶粉：翠绿色细粉末，清苦回甘，茶多酚提神，适合办公族醒脑。",
    "石榴汁：深红透亮液体，酸甜清爽，抗氧化能力强，适合抗初老人群。",
    "亚麻籽油：金黄透明油体，淡淡坚果香，Omega-3植物来源，适合素食补脑。",
    "无花果干：琥珀色扁圆果干，蜜甜柔韧，润肠通便，适合消化不良者。",
    "黑芝麻丸：乌黑油亮球粒，焦香浓郁，补铁养发，适合脱发困扰者。"
]

# 向量化并插入商品数据集
from tqdm import tqdm
doc_embeddings = embedding_model.encode(products)
data = []
for i, line in enumerate(tqdm(products, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})
milvus_client.insert(collection_name=collection_name, data=data)

/root/miniconda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Creating embeddings: 100%|██████████| 20/20 [00:00<00:00, 74965.22it/s]


{'insert_count': 20, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'cost': 0}

In [6]:
import random # 用于模拟生成表情
import time # 用于模拟网络延迟

def query_product_database(product_name: str) -> str:
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=embedding_model.encode(
            [product_name]
        ),  # 将商品名称转换为嵌入向量
        limit=1,  # 返回第1个结果
        search_params={"metric_type": "IP", "params": {}},  # 内积距离
        output_fields=["text"],  # 返回 text 字段
    )

    product_desc = search_res[0][0]["entity"]["text"]
    if not product_desc:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"
    else:
        return product_desc

## 4. 实战：构建小红书文案生成 Agent

In [7]:
import re

def remove_think_content(original_response: str) -> str:
    return re.sub(r'<think>.*?</think>', '', original_response, flags=re.DOTALL)

In [8]:
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    
    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。
        
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """
    
    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")

    product_desc = query_product_database(product_name)
    print(f"\n查询数据库中产品描述：{product_desc}\n")
    
    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"},
        {"role": "user", "content": f"产品描述信息如下：{product_desc}"}
    ]
    
    iteration_count = 0
    final_response = None
    
    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")
        
        try:
            # 调用 API，传入对话历史和工具定义
            response = client.chat.completions.create(
                model="deepseek-r1:8b",
                messages=messages
            )

            response_message = remove_think_content(response.choices[0].message.content)
            print(f"[模型生成结果] {response_message}")
                
            # --- START: 添加 JSON 提取和解析逻辑 ---
            json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message, re.DOTALL)
            
            if json_string_match:
                extracted_json_content = json_string_match.group(1)
                try:
                    final_response = json.loads(extracted_json_content)
                    print("Agent: 任务完成，成功解析最终JSON文案。")
                    return json.dumps(final_response, ensure_ascii=False, indent=2)
                except json.JSONDecodeError as e:
                    print(f"Agent: 提取到JSON块但解析失败: {e}")
                    print(f"尝试解析的字符串:\n{extracted_json_content}")
        except Exception as e:
            print(f"调用 API 时发生错误: {e}")
            break    
    return "未能成功生成文案。"

## 5. 实际测试与文案生成

现在，让我们调用我们构建的 `generate_rednote` 函数，看看它能生成什么样的爆款文案！

In [9]:
import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [10]:
# 测试案例 1
product_name_1 = "三文鱼"
tone_style_1 = "自然生态"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)
print("--- 格式化后的小红书文案 1 ---")
print(format_rednote_for_markdown(result_1))
# print(format_rednote_for_markdown(remove_think_content(result_1)))


🚀 启动小红书文案生成助手，产品：三文鱼，风格：自然生态


查询数据库中产品描述：三文鱼刺身：橘红色鲜嫩鱼片，冰凉滑嫩，富含Omega-3，适合心脑血管保养者。

-- Iteration 1 --
[模型生成结果] 
```json
{
  "title": "#一口惊艳！橘红色的三文鱼刺身绝了 🍱✨",
  "body": "真的被这个颜色打到了！🍊\n\n切开的一瞬间，冰凉柔嫩、透着粉橙色的肉质简直美炸了。咬下去第一口就知道这是什么神仙口感～\n\n清淡中带着一丝甜味，肥而不腻入口即化✨而且富含超级健康的Omega-3🥰心脑血管保养就靠它了！",
  "hashtags": ["#美食测评", "#健康饮食", "#三文鱼刺身", "#生食爱好者", "#每日养生必买"],
  "emojis": ["鲑鱼", "薄荷叶", "美味冰", "嫩牛排", "心形"]
}
```

**注释：**  
1. **标题策略**：直接点出视觉冲击力（橘红色），使用emoji增强吸引力（鲑鱼emoji）。核心关键词「三文鱼刺身」置于中间，降低话题门槛感。
2. **正文逻辑优化**：
 - 第一段解决新鲜与颜值痛点
 - 第二段呈现多重感官体验，穿插饮食场景化表达 
 - 最后强调营养价值并植入生活方式标签“精致养生活”
3. 标签选择兼顾产品关键词与生活方式延伸：“#每日养生必买”贴合心脑血管保养定位，“#精致生活”提升内容质感。
Agent: 任务完成，成功解析最终JSON文案。

--- 生成的文案 1 ---
{
  "title": "#一口惊艳！橘红色的三文鱼刺身绝了 🍱✨",
  "body": "真的被这个颜色打到了！🍊\n\n切开的一瞬间，冰凉柔嫩、透着粉橙色的肉质简直美炸了。咬下去第一口就知道这是什么神仙口感～\n\n清淡中带着一丝甜味，肥而不腻入口即化✨而且富含超级健康的Omega-3🥰心脑血管保养就靠它了！",
  "hashtags": [
    "#美食测评",
    "#健康饮食",
    "#三文鱼刺身",
    "#生食爱好者",
    "#每日养生必买"
  ],
  "emojis": [
    "鲑鱼",
    "薄荷叶",
    "美味冰",
    "嫩牛排",
    "心形"
  ]
}
--- 格式化

In [11]:
# 测试案例 2
product_name_2 = "巧克力"
tone_style_2 = "历史传承"
result_2 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 2 ---")
print(result_2)
print("--- 格式化后的小红书文案 2 ---")
print(format_rednote_for_markdown(result_2))
# print(format_rednote_for_markdown(remove_think_content(result_2)))


🚀 启动小红书文案生成助手，产品：巧克力，风格：历史传承


查询数据库中产品描述：黑巧克力（85%可可）：深褐色光滑块状，微苦带甜，富含抗氧化剂，适合需要提神和控糖的人群。

-- Iteration 1 --
[模型生成结果] 
```json
{
  "title": "舌尖上的历史传承：品味黑巧85%，唤醒你的味蕾记忆",
  "body": "姐妹们！今天我要带你们开启一场穿越时空的味觉之旅～🧐 深褐色的巧克力块，光滑如丝绸，每一口都仿佛在品味千年来的传统美食精髓✨\n\n这款黑巧含可可是的85%哦！微苦中带着丝丝甜意💖 相比那些甜蜜负担的小零食，这块暗黑系小宝贝才是真正给身体加分的秘密武器🍵 既能让现代忙碌人提神醒脑💪 而且对控糖党我超友好🌿\n\n重点来了！巧克力不只是零食，更是高颜值生活好物🎨 复古红与经典黑组合起来，随手放桌面或收纳柜都能凹出高级感🌹 正是这种传统工艺的匠心坚持才让它如此珍贵💫～ 走进精品巧克力的世界吧！",
  "hashtags": ["#黑巧控糖","#巧克力历史传承","#精致生活","#品味时光","#舌尖记忆"],
  "emojis": ["🍫", "🍵", "💪", "🌹", "✨"]
}
```
Agent: 任务完成，成功解析最终JSON文案。

--- 生成的文案 2 ---
{
  "title": "舌尖上的历史传承：品味黑巧85%，唤醒你的味蕾记忆",
  "body": "姐妹们！今天我要带你们开启一场穿越时空的味觉之旅～🧐 深褐色的巧克力块，光滑如丝绸，每一口都仿佛在品味千年来的传统美食精髓✨\n\n这款黑巧含可可是的85%哦！微苦中带着丝丝甜意💖 相比那些甜蜜负担的小零食，这块暗黑系小宝贝才是真正给身体加分的秘密武器🍵 既能让现代忙碌人提神醒脑💪 而且对控糖党我超友好🌿\n\n重点来了！巧克力不只是零食，更是高颜值生活好物🎨 复古红与经典黑组合起来，随手放桌面或收纳柜都能凹出高级感🌹 正是这种传统工艺的匠心坚持才让它如此珍贵💫～ 走进精品巧克力的世界吧！",
  "hashtags": [
    "#黑巧控糖",
    "#巧克力历史传承",
    "#精致生活",
    "#品味时光",
    "#舌尖记忆"
  ],
  "emojis": [
    "🍫",
    "🍵",
 

In [12]:
# 测试案例 3
product_name_3 = "椰子水"
tone_style_3 = "清爽健康"
result_3 = generate_rednote(product_name_3, tone_style_3)

print("\n--- 生成的文案 3 ---")
print(result_3)
print("--- 格式化后的小红书文案 3 ---")
print(format_rednote_for_markdown(result_3))
# print(format_rednote_for_markdown(remove_think_content(result_3)))


🚀 启动小红书文案生成助手，产品：椰子水，风格：清爽健康


查询数据库中产品描述：椰子水：透明清亮液体，清甜爽口，电解质丰富，适合运动后补水。

-- Iteration 1 --
[模型生成结果] 
{
  "title": "椰子水拯救者！运动后喝一口鲜榨电解质，身体瞬间满分💯",
  "body": "姐妹们，最近我发现了一个超级清爽的补水神器——椰子水！🌿 它一打开就是那种透明清亮的惊喜感，喝下去的第一口感觉像夏天的第一缕凉风，清甜到让人忘记了所有烦恼～💦 动作爱好者都知道，运动出汗可不是闹着玩的，这时候电解质水分流失，直接导致疲劳和肌肉酸痛。别担心！椰子水富含天然电解质和矿物质，帮我快速恢复体力，保持水分平衡，再也不怕多汗日了。它不仅仅是饮品那么简单，还能提升你的能量水平，让你活力满满去面对每天的挑战～我最爱在健身房后立马来一瓶，或者在炎炎夏日配着户外运动解渴。💖 你们试过椰子水吗？快分享给好友吧！#健康生活从细节开始🔥 #我的运动日常 #天然电解质益处",
  "hashtags": ["#coconutwater", "#healthyhydration", "#sportslife", "#nativenatural", "#summerrefresher"],
  "emojis": ["✨", "💧", "🌿", "💯", "🍋"]
}
-- Iteration 2 --
[模型生成结果] 
{
  "title": "椰汁暴击！运动后这瓶清新神器让你活力满分💥",
  "body": " Hey宝宝们~🌞 最近天气热到爆，你们有没有发现一款超级清爽的饮品救星？我是安利椰子水！🌿它就像大自然送来的甘露，透明清亮、入口清甜爽口～💦 每当运动完一身大汗，喝一口就能瞬间回血补状态，富含电解质的营养宝，补水效果杠杠哒！💪 我个人超爱那些无添加品牌的椰子水（比如XX牌子），小红书姐妹们都说它比汽水健康N倍哦～✨ 小tips：买散装时认准新鲜椰汁，记得冷藏更解渴！夏天必备清单里加一瓶吧🌈。",
  "hashtags": ["#椰子水", "#运动补水", "#自然好物", "#健康生活", "#清凉夏日"],
  "emojis": ["🌞", "🧃", "💪", "✨", "🌈"]
}
-- Iteration 3 --
[模型生成结果] 
{
  